Missing number in Python is represented as nan.

I can detect which cells have missing values, and then count how many there are in each column with the command:
    
print(data.isnull().sum())

# Solution 1: Drop Columns with Missing Values

Drop columns with missing values can be useful when most values in a column are missing. (original_data is a data frame)

data_without_missing_values = original_data.dropna(axis=1)

I want to drop the same column in both training and test sets.

cols_with_missing = [col for col in original_data.columns 
                                 if original_data[col].isnull().any()]
                                 
redued_original_data = original_data.drop(cols_with_missing, axis=1)

reduced_test_data = test_data.drop(cols_with_missing, axis=1)

# Solution 2: Imputation

Fill in the missing value with some number

In [11]:
# The default behavior fills in the mean value for imputation
from sklearn.preprocessing import Imputer

my_imputer = Imputer()

data_with_imputed_values = my_imputer.fit_transform(original_data)

# Solution 3: An Extension To Imputation

Make copy to avoid changing original data when Imputing. However, the effectiveness can be dataset sepecific and could vary case by case.

new_data = original_data.copy()

Make new columns indicating what will be imputed

cols_with_missing = (col for col in new_data.columns 

                                 if new_data[c].isnull().any())
                                 
for col in cols_with_missing:

    new_data[col + '_was_missing'] = new_data[col].isnull()

Imputation

my_imputer = Imputer()

new_data = my_imputer.fit_transform(new_data)

# Example: Melbourne Housing data

Predict housing prices from the Melbourne Housing data

Load data

In [31]:
import pandas as pd

In [32]:
melb_data = pd.read_csv('melb_data.csv')

Define target variable

In [33]:
melb_target = melb_data.Price

Drop missing values in the target variable: price

In [34]:
melb_predictors = melb_data.drop(['Price'], axis=1)

To keep the example simple, I will use only numeric predictors

In [35]:
melb_numeric_predictors = melb_predictors.select_dtypes(exclude=['object'])

Measure Quality of an Approach

Split data into training and test sets

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
X_train, X_test, y_train, y_test = train_test_split(melb_numeric_predictors, 
                                                    melb_target, 
                                                    
                                                    train_size=0.7, 
                                                    test_size=0.3, 
                                                    random_state=0)

Report the out-of-sample mean absolute error (MAE) score from a RandomForest

In [38]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [39]:
def score_dataset(X_train, X_test, y_train, y_test):
    # use random forest model
    model = RandomForestRegressor()
    
    # fit model
    model.fit(X_train, y_train)
    
    # get predictions
    preds = model.predict(X_test)
    
    # compute MAE score
    return mean_absolute_error(y_test, preds)

Get Model Score from Dropping Columns with Missing Values

In [40]:
# search for the missing columns
cols_with_missing = [col for col in X_train.columns 
                                 if X_train[col].isnull().any()]

# drop the found columns in training and test sets
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_test  = X_test.drop(cols_with_missing, axis=1)

In [41]:
print("Mean Absolute Error from dropping columns with Missing Values:")
print(score_dataset(reduced_X_train, reduced_X_test, y_train, y_test))

Mean Absolute Error from dropping columns with Missing Values:
350023.275539


Get Model Score from Imputation

In [42]:
from sklearn.preprocessing import Imputer

In [43]:
my_imputer = Imputer()

In [44]:
# define imputed indicator matrices from training and test sets
imputed_X_train = my_imputer.fit_transform(X_train)
imputed_X_test = my_imputer.transform(X_test)

In [45]:
print("Mean Absolute Error from Imputation:")
print(score_dataset(imputed_X_train, imputed_X_test, y_train, y_test))

Mean Absolute Error from Imputation:
203996.147726


Get Score from Imputation with Extra Columns Showing What Was Imputed

In [47]:
# Make copies
imputed_X_train_plus = X_train.copy()
imputed_X_test_plus = X_test.copy()

In [48]:
# Make new columns indicating what will be imputed
cols_with_missing = (col for col in X_train.columns # find the missing columns in X_train
                                 if X_train[col].isnull().any())

for col in cols_with_missing:
    imputed_X_train_plus[col + '_was_missing'] = imputed_X_train_plus[col].isnull() # concatenation
    imputed_X_test_plus[col + '_was_missing'] = imputed_X_test_plus[col].isnull()

In [49]:
# Imputation
my_imputer = Imputer()

imputed_X_train_plus = my_imputer.fit_transform(imputed_X_train_plus) # use copies instead of original 
imputed_X_test_plus = my_imputer.transform(imputed_X_test_plus)

In [50]:
print("Mean Absolute Error from Imputation while Track What Was Imputed:")
print(score_dataset(imputed_X_train_plus, imputed_X_test_plus, y_train, y_test))

Mean Absolute Error from Imputation while Track What Was Imputed:
201750.410545


Comment: In this example, the imputation and extension to imputation methods do not show much a difference.

# Example: Iowa Housing data

In [54]:
# import data
import pandas as pd

main_file_path = 'train.csv'
iowa_data = pd.read_csv(main_file_path)

In [55]:
# define predictor matrix and target variable
iowa_target = data.SalePrice 

# drop missing values in the target variable
iowa_predictors = iowa_data.drop(['SalePrice'], axis=1)

# only use numeric values
iowa_numeric_predictors = iowa_predictors.select_dtypes(exclude=['object'])

In [56]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(iowa_numeric_predictors, 
                                                    iowa_target, # actual
                                                    
                                                    train_size=0.7, 
                                                    test_size=0.3, 
                                                    random_state=0)

In [57]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_test, y_train, y_test): # the first two arguments will vary across the three methods below
    # use random forest model
    model = RandomForestRegressor()
    
    # fit model
    model.fit(X_train, y_train)
    
    # get predictions
    preds = model.predict(X_test)
    
    # compute MAE score
    return mean_absolute_error(y_test, preds)

Method 1: drop columns with missing values

In [58]:
# identify the missing columns
cols_with_missing = [col for col in X_train.columns 
                                 if X_train[col].isnull().any()]

# drop the above columns in training and test sets
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_test  = X_test.drop(cols_with_missing, axis=1)

# print out result
print("Mean Absolute Error from dropping columns with Missing Values:")
print(score_dataset(reduced_X_train, reduced_X_test, y_train, y_test))

Mean Absolute Error from dropping columns with Missing Values:
19153.639726


The Iowa data doesn't have a lot of columns with missing values

In [59]:
cols_with_missing

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

Method 2: Imputation

In [60]:
from sklearn.preprocessing import Imputer # default: filled with mean values

my_imputer = Imputer()

# define imputed indicator matrices from training and test sets
imputed_X_train = my_imputer.fit_transform(X_train)
imputed_X_test = my_imputer.transform(X_test)

print("Mean Absolute Error from dropping columns with Missing Values:")
print(score_dataset(reduced_X_train, reduced_X_test, y_train, y_test))

Mean Absolute Error from dropping columns with Missing Values:
18598.6949772


Method 3: an extension to imputation

In [61]:
# make copies
imputed_X_train_plus = X_train.copy()
imputed_X_test_plus = X_test.copy()

# make new columns indicating what will be imputed
cols_with_missing = (col for col in X_train.columns # find the missing columns in X_train
                                 if X_train[col].isnull().any())

for col in cols_with_missing:
    imputed_X_train_plus[col + '_was_missing'] = imputed_X_train_plus[col].isnull() # concatenation
    imputed_X_test_plus[col + '_was_missing'] = imputed_X_test_plus[col].isnull()
    
# imputation
my_imputer = Imputer()

imputed_X_train_plus = my_imputer.fit_transform(imputed_X_train_plus) # use copies instead of original 
imputed_X_test_plus = my_imputer.transform(imputed_X_test_plus)

print("Mean Absolute Error from Imputation while Track What Was Imputed:")
print(score_dataset(imputed_X_train_plus, imputed_X_test_plus, y_train, y_test))

Mean Absolute Error from Imputation while Track What Was Imputed:
19081.7216895


Comment: 

Decision tree model yields an MAE of 32707.7178082. Random forest model improves the MAE to 23674.6314025. The score is improved with the above three methods.